
# RAG 실습



## 0) 설치 & 환경 설정

- Retrieval: sentence-transformers, faiss-cpu, rank-bm25
- Generation: transformers
- Framework: langchain-*
- Compression: llmlingua
- Evaluation: ragas, datasets
-  openai key

In [1]:

!pip -q install sentence-transformers faiss-cpu rank-bm25 transformers langchain-text-splitters
!pip -q install "langchain==0.2.14" "langchain-community==0.2.12" "langchain-core>=0.2.0"
!pip -q install llmlingua
!pip -q install ragas datasets
!pip -q install openai

import random, numpy as np
random.seed(42); np.random.seed(42)



## 1) 금융 코퍼스(토이) & 라벨

간단 문단 텍스트 + 라벨(topic, tags) 구성
나중에 자신의 도메인 문서(사내 위키/FAQ/리포트)로 교체 가능


In [1]:

finance_docs = [
    ("삼성전자 메모리 부문은 DDR5 전환과 HBM 수요 확대에 힘입어 수익성이 회복되고 있다.", {"topic":"samsung", "tags":["memory","hbm","profitability"]}),
    ("SK하이닉스는 HBM3E 양산을 통해 엔비디아와의 파트너십을 강화하고 매출 다변화를 꾀한다.", {"topic":"skhynix", "tags":["hbm","nvidia","revenue"]}),
    ("엔비디아는 데이터센터용 GPU 출하가 확대되어 분기 최대 실적을 갱신했다.", {"topic":"nvidia", "tags":["gpu","datacenter","earnings"]}),
    ("TSMC는 3나노 공정 수율 개선으로 하이엔드 고객 주문을 확보하며 파운드리 점유율을 높였다.", {"topic":"tsmc", "tags":["foundry","3nm","yield"]}),
    ("애플은 A시리즈 칩 자체 설계를 강화하고 TSMC와의 공조로 성능과 전력 효율을 개선한다.", {"topic":"apple", "tags":["iphone","asoc","tsmc"]}),
    ("미국의 기준금리 동결 기조가 지속될 경우, 성장주 밸류에이션은 점진적 회복세를 보일 수 있다.", {"topic":"macro", "tags":["rates","valuation","growth"]}),
    ("원/달러 환율 상승은 반도체 수출 채산성에 단기적으로 긍정적 영향을 준다.", {"topic":"fx", "tags":["krw","usd","semiconductor","exports"]}),
    ("HBM 채택 확대는 LLM 추론 효율과 메모리 대역폭 병목을 동시에 개선한다.", {"topic":"hbm", "tags":["bandwidth","llm","inference"]}),
    ("반도체 공급망 다변화는 지정학적 리스크를 관리하는 핵심 전략으로 자리 잡고 있다.", {"topic":"supplychain", "tags":["geopolitics","risk","diversification"]}),
    ("서버 DRAM 수요는 AI 워크로드 증가와 함께 중장기 성장 추세가 지속될 전망이다.", {"topic":"drams", "tags":["server","ai","demand"]}),
]
docs_texts = [d[0] for d in finance_docs]
docs_labels = [d[1] for d in finance_docs]
print(f"#문단 수: {len(docs_texts)}")


#문단 수: 10



## 2) 질의 세트 & 정답 기준(needs)

각 질의에 대해 needs = {topic, tags} 를 설정하여 관련성 판정에 사용
실제 과제에서는 별도의 정답 문장(ground truth)을 마련하는 것을 권장


In [2]:

queries = [
    {"q": "HBM이 LLM 추론 성능에 주는 영향과 관련된 근거 찾아줘", "needs": {"tags":["hbm","llm","inference"]}},
    {"q": "엔비디아의 GPU 수요와 데이터센터 실적 관련 배경을 설명해줘", "needs": {"topic":["nvidia"], "tags":["gpu","datacenter","earnings"]}},
    {"q": "삼성전자와 하이닉스의 HBM 전략 차이를 비교해줘", "needs": {"topic":["samsung","skhynix"], "tags":["hbm"]}},
    {"q": "미국 금리와 성장주 밸류에이션의 관계를 보여주는 문맥 찾아줘", "needs": {"topic":["macro"], "tags":["rates","valuation","growth"]}},
    {"q": "환율 변화가 반도체 수출에 미치는 영향", "needs": {"topic":["fx"], "tags":["krw","usd","semiconductor","exports"]}},
]
print(f"#질의 수: {len(queries)}")


#질의 수: 5



## 3) 청킹(Chunking): Manual vs LangChain

- Manual: NLTK 문장 분리 -> 슬라이딩 윈도우(Overlap)
- LangChain: RecursiveCharacterTextSplitter 로 길이 기반 분할


In [3]:
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [4]:

import nltk, itertools
nltk.download('punkt')

def sent_chunk(text):
    "한 문단을 문장 단위로 분리"
    return nltk.sent_tokenize(text)

def sliding_window(sents, w=2, overlap=1):
    """
    문장 리스트(sents)를 길이 w로 겹치게 분할
    overlap=1이면 이전 청크와 1문장 겹침
    """
    chunks=[]; step=max(1, w-overlap)
    for i in range(0, max(1, len(sents)-w+1), step):
        chunks.append(" ".join(sents[i:i+w]))
    return chunks

chunks_manual = list(itertools.chain.from_iterable(sliding_window(sent_chunk(d), w=2, overlap=1) for d in docs_texts))
print("Manual chunks 예시:\n", "\n---\n".join(chunks_manual[:3]))


Manual chunks 예시:
 삼성전자 메모리 부문은 DDR5 전환과 HBM 수요 확대에 힘입어 수익성이 회복되고 있다.
---
SK하이닉스는 HBM3E 양산을 통해 엔비디아와의 파트너십을 강화하고 매출 다변화를 꾀한다.
---
엔비디아는 데이터센터용 GPU 출하가 확대되어 분기 최대 실적을 갱신했다.


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:

from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=220, chunk_overlap=40, separators=["\n\n", "\n", " ", ""],
)
chunks_lc = text_splitter.split_text("\n\n".join(docs_texts))
print("LangChain chunks 예시:\n", "\n---\n".join(chunks_lc[:3]))


LangChain chunks 예시:
 삼성전자 메모리 부문은 DDR5 전환과 HBM 수요 확대에 힘입어 수익성이 회복되고 있다.

SK하이닉스는 HBM3E 양산을 통해 엔비디아와의 파트너십을 강화하고 매출 다변화를 꾀한다.

엔비디아는 데이터센터용 GPU 출하가 확대되어 분기 최대 실적을 갱신했다.

TSMC는 3나노 공정 수율 개선으로 하이엔드 고객 주문을 확보하며 파운드리 점유율을 높였다.
---
애플은 A시리즈 칩 자체 설계를 강화하고 TSMC와의 공조로 성능과 전력 효율을 개선한다.

미국의 기준금리 동결 기조가 지속될 경우, 성장주 밸류에이션은 점진적 회복세를 보일 수 있다.

원/달러 환율 상승은 반도체 수출 채산성에 단기적으로 긍정적 영향을 준다.

HBM 채택 확대는 LLM 추론 효율과 메모리 대역폭 병목을 동시에 개선한다.
---
반도체 공급망 다변화는 지정학적 리스크를 관리하는 핵심 전략으로 자리 잡고 있다.

서버 DRAM 수요는 AI 워크로드 증가와 함께 중장기 성장 추세가 지속될 전망이다.



## 4) 쿼리 리라이팅/분해(규칙 기반)

복합 질의를 의도별 서브쿼리로 분해 -> 각 서브쿼리 검색 결과를 max-pool 로 합성
규칙 예시: 기업 비교(삼성전자 vs 하이닉스), HBM/LLM/GPU/금리/환율 키워드 확장


In [9]:
# OpenAI API를 사용해 질의를 서브쿼리로 분해하는 예시
# 사전 준비: `pip install openai` (최신 SDK), 환경변수 OPENAI_API_KEY 설정

import os, json, time
from typing import List
from openai import OpenAI, APIError, RateLimitError

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))


SYSTEM_PROMPT = (
    "당신은 정보검색 어시스턴트입니다. 사용자의 한국어 질의를 "
    "검색 친화적인 여러 '서브쿼리'로 분해합니다. "
    "반드시 JSON 배열(list of strings)만 출력하세요. "
    "예: [\"HBM LLM 추론 성능 영향\", \"메모리 대역폭 병목\"]"
)

def decompose_with_openai(q: str, model: str = "gpt-4o-mini") -> List[str]:
    """
    OpenAI 모델로 질의를 검색용 서브쿼리 리스트로 분해.
    실패 시 원문 질의만 담긴 리스트를 반환.
    """
    for attempt in range(3):
        try:
            resp = client.chat.completions.create(
                model=model,
                temperature=0,
                messages=[
                    {"role": "system", "content": SYSTEM_PROMPT},
                    {
                        "role": "user",
                        "content": (
                            "질의:\n"
                            f"{q}\n\n"
                            "지침:\n"
                            "- 비교/대조 의도가 보이면 각 엔티티별 서브쿼리와 'A vs B' 서브쿼리를 함께 만드세요.\n"
                            "- 금융 도메인 키워드(HBM, LLM, GPU, 금리, 환율 등)는 구체화하세요.\n"
                            "- 반드시 JSON 배열만 출력하세요."
                        ),
                    },
                ],
            )
            text = resp.choices[0].message.content.strip()
            # 모델이 코드블록으로 감싸거나 주석을 붙이는 경우를 대비해 JSON만 추출 시도
            start = text.find("[")
            end = text.rfind("]")
            if start != -1 and end != -1:
                text = text[start : end + 1]
            subs = json.loads(text)
            # 품질 필터: 문자열만 남기고 공백/중복 제거
            subs = [s.strip() for s in subs if isinstance(s, str) and s.strip()]
            # 최소 한 개는 보장
            return subs if subs else [q]
        except (json.JSONDecodeError, APIError, RateLimitError) as e:
            # 간단한 백오프 후 재시도
            time.sleep(0.8 * (attempt + 1))
    return [q]


def rule_rewrite(q: str) -> List[str]:
    subs = decompose_with_openai(q)
    # 원문 포함 + 중복 제거 (집합 보존 순서)
    seen, out = set(), []
    for s in [q] + subs:
        if s not in seen:
            seen.add(s)
            out.append(s)
    return out

# 데모: queries 리스트를 순회하며 분해 결과 출력
for ex in queries:
    print(ex["q"], "->", rule_rewrite(ex["q"]))


HBM이 LLM 추론 성능에 주는 영향과 관련된 근거 찾아줘 -> ['HBM이 LLM 추론 성능에 주는 영향과 관련된 근거 찾아줘', 'HBM LLM 추론 성능 영향', 'HBM vs DDR 메모리 성능 비교', 'LLM 추론 성능 최적화 방법', 'HBM 메모리 대역폭', 'GPU 성능과 HBM의 관계', 'LLM 모델 크기와 HBM의 영향']
엔비디아의 GPU 수요와 데이터센터 실적 관련 배경을 설명해줘 -> ['엔비디아의 GPU 수요와 데이터센터 실적 관련 배경을 설명해줘', '엔비디아 GPU 수요', '엔비디아 데이터센터 실적', 'GPU 수요 증가 원인', '데이터센터 실적 향상 요인', '엔비디아 GPU vs AMD GPU', '엔비디아 데이터센터 실적 vs 경쟁사 실적']
삼성전자와 하이닉스의 HBM 전략 차이를 비교해줘 -> ['삼성전자와 하이닉스의 HBM 전략 차이를 비교해줘', '삼성전자 HBM 전략', '하이닉스 HBM 전략', '삼성전자와 하이닉스 HBM 전략 비교', 'HBM 기술 차이 삼성전자 하이닉스', 'HBM 시장 점유율 삼성전자 하이닉스']
미국 금리와 성장주 밸류에이션의 관계를 보여주는 문맥 찾아줘 -> ['미국 금리와 성장주 밸류에이션의 관계를 보여주는 문맥 찾아줘', '미국 금리와 성장주 밸류에이션 관계', '미국 금리 변화가 성장주에 미치는 영향', '성장주 밸류에이션과 금리의 상관관계', '금리 인상과 성장주 가치 평가', '미국 금리 vs 성장주 밸류에이션', '금리와 주식 시장의 상관관계', '성장주와 금리 변화의 상호작용']
환율 변화가 반도체 수출에 미치는 영향 -> ['환율 변화가 반도체 수출에 미치는 영향', '환율 변화 반도체 수출 영향', '환율 변화 HBM 반도체 수출', '환율 변화 LLM 반도체 수출', '환율 변화 GPU 반도체 수출', '환율 변화 금리 반도체 수출', '환율 상승 반도체 수출 감소 vs 환율 하락 반도체 수출 증가']



## 5) 임베딩 모델 로드 (옵션 NMIXX -> MiniLM)

아래 NMIXX_HUB_ID 에 실제 레포 ID를 넣으면 NMIXX와 비교 가능


In [10]:
from sentence_transformers import SentenceTransformer
import numpy as np, faiss, time

NMIXX_HUB_ID = "nmixx-fin/nmixx-bge-m3"

def load_any(try_list):
    "여러 후보를 순서대로 로드 시도 -> 성공 시 즉시 반환"
    last_err = None
    for name in try_list:
        try:
            print("[LOAD]", name)
            start = time.time()
            m = SentenceTransformer(name)
            print(" -> ok in %.1fs" % (time.time()-start))
            return name, m
        except Exception as e:
            last_err = e
            print(" -> failed:", e)
    raise last_err

model_candidates = [NMIXX_HUB_ID, "sentence-transformers/all-MiniLM-L6-v2"]
model_name, embed_model = load_any(model_candidates)
print(">>> 사용 임베딩:", model_name)

[LOAD] nmixx-fin/nmixx-bge-m3


No sentence-transformers model found with name nmixx-fin/nmixx-bge-m3. Creating a new one with mean pooling.


 -> ok in 5.5s
>>> 사용 임베딩: nmixx-fin/nmixx-bge-m3



## 6) 검색 구현: Manual vs LangChain

- Manual
  - Vector: FAISS(IndexFlatIP) + 코사인 유사도(정규화 내적)
  - BM25: rank-bm25
  - Hybrid: 정규화 후 가중합 (lam*BM25 + (1-lam)*cosine)
- LangChain
  - FAISS VectorStore, BM25Retriever, EnsembleRetriever


In [11]:

def build_faiss(emb_model, texts):
    "텍스트 -> 임베딩 -> FAISS IndexFlatIP 빌드"
    vec = emb_model.encode(texts, convert_to_numpy=True, normalize_embeddings=True)
    index = faiss.IndexFlatIP(vec.shape[1]); index.add(vec)
    return index, vec

index_manual, vec_manual = build_faiss(embed_model, docs_texts)

from rank_bm25 import BM25Okapi
import nltk
nltk.download('punkt')
tokenized = [nltk.word_tokenize(t) for t in docs_texts]
bm25 = BM25Okapi(tokenized)

def cosine_scores(emb_model, index, query):
    "쿼리 하나에 대해 전체 문서 코사인 유사도 점수 배열 반환"
    qv = emb_model.encode([query], convert_to_numpy=True, normalize_embeddings=True)
    D,I = index.search(qv, len(docs_texts))
    scores = np.zeros(len(docs_texts)); scores[I[0]] = D[0]
    return scores

def bm25_scores(query):
    "BM25 점수 배열 반환"
    return bm25.get_scores(nltk.word_tokenize(query))

def hybrid_scores(emb_model, index, query, lam=0.4):
    "정규화된 가중합으로 BM25/코사인 결합"
    cos = cosine_scores(emb_model, index, query)
    bm  = bm25_scores(query)
    def norm(x): x = x.astype(np.float64); return (x - x.min())/(x.max()-x.min()+1e-9)
    return lam*norm(bm) + (1-lam)*norm(cos)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [36]:
# LangChain 구현 (개념 매핑)
from langchain_community.vectorstores import FAISS as LC_FAISS
from langchain_community.embeddings import SentenceTransformerEmbeddings
from langchain_community.retrievers import BM25Retriever
from langchain.retrievers.ensemble import EnsembleRetriever  

emb_lc = SentenceTransformerEmbeddings(model_name=model_name)
vs = LC_FAISS.from_texts(docs_texts, embedding=emb_lc)
retriever_vec = vs.as_retriever(search_type="similarity", search_kwargs={"k": 5})

bm25_lc = BM25Retriever.from_texts(docs_texts)
bm25_lc.k = 5

hybrid_ret = EnsembleRetriever(
    retrievers=[bm25_lc, retriever_vec],
    weights=[0.4, 0.6],
)

print("[LangChain Hybrid top-3]")
print(f"query: {queries[0]}")
for d in hybrid_ret.get_relevant_documents(queries[0]["q"])[:1]:
    print("-", d.page_content)


No sentence-transformers model found with name nmixx-fin/nmixx-bge-m3. Creating a new one with mean pooling.


[LangChain Hybrid top-3]
query: {'q': 'HBM이 LLM 추론 성능에 주는 영향과 관련된 근거 찾아줘', 'needs': {'tags': ['hbm', 'llm', 'inference']}}
- HBM 채택 확대는 LLM 추론 효율과 메모리 대역폭 병목을 동시에 개선한다.



## 7) 평가 지표 계산기 (P@k / R@k / MRR / nDCG@k)

토이 라벨(topic/tags) 기반 이진 판정


In [14]:

def is_relevant(doc_meta, needs):
    "문서 메타와 needs(topic/tags)를 바탕으로 관련성 판정"
    ok = False
    if "topic" in needs and doc_meta["topic"] in set(needs["topic"]):
        ok = True
    if "tags" in needs and set(needs["tags"]).intersection(set(doc_meta["tags"])):
        ok = True
    return ok

def topk_by_scores(scores, k=5):
    "점수 배열에서 상위 k개 인덱스/점수 반환"
    order = np.argsort(scores)[::-1][:k]
    return order, scores[order]

def metrics_at_k(ranked_idx, needs, k=5):
    "P@k, R@k, MRR, nDCG@k 계산"
    rel = [1 if is_relevant(docs_labels[i], needs) else 0 for i in ranked_idx[:k]]
    p_at_k = sum(rel)/k
    total_rel = sum(1 for i in range(len(docs_texts)) if is_relevant(docs_labels[i], needs))
    r_at_k = sum(rel)/max(1, total_rel)
    # MRR
    mrr = 0.0
    for rank, r in enumerate(rel, start=1):
        if r==1: mrr = 1.0/rank; break
    # nDCG@k
    import math
    dcg = sum((rel[i]/math.log2(i+2)) for i in range(len(rel)))
    ideal = sorted(rel, reverse=True)
    idcg = sum((ideal[i]/math.log2(i+2)) for i in range(len(ideal)))
    ndcg = dcg/(idcg+1e-9)
    return {"P@%d"%k: p_at_k, "R@%d"%k: r_at_k, "MRR": mrr, "nDCG@%d"%k: ndcg}



## 8) 통합 평가 루프 (Rewrite + Vector/BM25/Hybrid)

서브쿼리별 점수를 max-pool 로 합성 -> 각 모드(Vector/BM25/Hybrid) 지표 계산
lam=0.3/0.5/0.7 변화에 따른 Hybrid 성능 변화를 확인


In [15]:

import pandas as pd
import numpy as np

def eval_query(qobj, lam_list=(0.3,0.5,0.7), k=5, use_rewrite=True):
    q = qobj["q"]; needs = qobj["needs"]
    subs = rule_rewrite(q) if use_rewrite else [q]
    vec = np.zeros(len(docs_texts)); bm = np.zeros(len(docs_texts))
    for sub in subs:
        vec = np.maximum(vec, cosine_scores(embed_model, index_manual, sub))
        bm  = np.maximum(bm, bm25_scores(sub))
    res = {}
    order,_ = topk_by_scores(vec, k=k); res["vector"] = metrics_at_k(order, needs, k=k)
    order,_ = topk_by_scores(bm, k=k);  res["bm25"]   = metrics_at_k(order, needs, k=k)
    for lam in lam_list:
        hyb = (
            lam * ((bm - bm.min()) / (np.ptp(bm) + 1e-9)) +
            (1 - lam) * ((vec - vec.min()) / (np.ptp(vec) + 1e-9))
        )
        order,_ = topk_by_scores(hyb, k=k); res[f"hybrid_{lam}"] = metrics_at_k(order, needs, k=k)
    return res, subs

rows = []
for qobj in queries:
    res, subs = eval_query(qobj, lam_list=(0.3,0.5,0.7), k=5, use_rewrite=True)
    for mode, m in res.items():
        row = {"query": qobj["q"], "rewrite_subqs": " | ".join(subs), "mode": mode}
        row.update(m)
        rows.append(row)
df_eval = pd.DataFrame(rows).sort_values(["query","mode"])
df_eval


,query,rewrite_subqs,mode,P@5,R@5,MRR,nDCG@5
1,HBM이 LLM 추론 성능에 주는 영향과 관련된 근거 찾아줘,HBM이 LLM 추론 성능에 주는 영향과 관련된 근거 찾아줘 | HBM LLM 추론...,bm25,0.4,0.666667,1.0,1.000000
2,HBM이 LLM 추론 성능에 주는 영향과 관련된 근거 찾아줘,HBM이 LLM 추론 성능에 주는 영향과 관련된 근거 찾아줘 | HBM LLM 추론...,hybrid_0.3,0.6,1.000000,1.0,0.946902
3,HBM이 LLM 추론 성능에 주는 영향과 관련된 근거 찾아줘,HBM이 LLM 추론 성능에 주는 영향과 관련된 근거 찾아줘 | HBM LLM 추론...,hybrid_0.5,0.6,1.000000,1.0,0.946902
4,HBM이 LLM 추론 성능에 주는 영향과 관련된 근거 찾아줘,HBM이 LLM 추론 성능에 주는 영향과 관련된 근거 찾아줘 | HBM LLM 추론...,hybrid_0.7,0.6,1.000000,1.0,0.946902
0,HBM이 LLM 추론 성능에 주는 영향과 관련된 근거 찾아줘,HBM이 LLM 추론 성능에 주는 영향과 관련된 근거 찾아줘 | HBM LLM 추론...,vector,0.6,1.000000,1.0,1.000000
16,미국 금리와 성장주 밸류에이션의 관계를 보여주는 문맥 찾아줘,미국 금리와 성장주 밸류에이션의 관계를 보여주는 문맥 찾아줘 | 미국 금리와 성장주...,bm25,0.2,1.000000,1.0,1.000000
17,미국 금리와 성장주 밸류에이션의 관계를 보여주는 문맥 찾아줘,미국 금리와 성장주 밸류에이션의 관계를 보여주는 문맥 찾아줘 | 미국 금리와 성장주...,hybrid_0.3,0.2,1.000000,1.0,1.000000
18,미국 금리와 성장주 밸류에이션의 관계를 보여주는 문맥 찾아줘,미국 금리와 성장주 밸류에이션의 관계를 보여주는 문맥 찾아줘 | 미국 금리와 성장주...,hybrid_0.5,0.2,1.000000,1.0,1.000000
19,미국 금리와 성장주 밸류에이션의 관계를 보여주는 문맥 찾아줘,미국 금리와 성장주 밸류에이션의 관계를 보여주는 문맥 찾아줘 | 미국 금리와 성장주...,hybrid_0.7,0.2,1.000000,1.0,1.000000
15,미국 금리와 성장주 밸류에이션의 관계를 보여주는 문맥 찾아줘,미국 금리와 성장주 밸류에이션의 관계를 보여주는 문맥 찾아줘 | 미국 금리와 성장주...,vector,0.2,1.000000,1.0,1.000000



## 9) 리랭커 (Cross-Encoder)

Hybrid 상위 후보를 교차인코더로 재정렬 -> 정밀도 향상 기대
수업에서는 경량 모델 권장: BAAI/bge-reranker-v2-m3


In [37]:

from sentence_transformers import CrossEncoder

demo_q = queries[0]
hyb = hybrid_scores(embed_model, index_manual, demo_q["q"], lam=0.5)
cand_idx = np.argsort(hyb)[::-1][:5]
cand = [docs_texts[i] for i in cand_idx]

rerank_model = "BAAI/bge-reranker-v2-m3" # reranker

rer = CrossEncoder(rerank_model)
pairs = [[demo_q["q"], c] for c in cand]
scores = rer.predict(pairs)
order2 = scores.argsort()[::-1]
cand_rr = [cand[i] for i in order2]
print(demo_q)
print("[Before]\n", "\n---\n".join(cand))
print("\n[After Rerank]\n", "\n---\n".join(cand_rr))



{'q': 'HBM이 LLM 추론 성능에 주는 영향과 관련된 근거 찾아줘', 'needs': {'tags': ['hbm', 'llm', 'inference']}}
[Before]
 HBM 채택 확대는 LLM 추론 효율과 메모리 대역폭 병목을 동시에 개선한다.
---
삼성전자 메모리 부문은 DDR5 전환과 HBM 수요 확대에 힘입어 수익성이 회복되고 있다.
---
원/달러 환율 상승은 반도체 수출 채산성에 단기적으로 긍정적 영향을 준다.
---
SK하이닉스는 HBM3E 양산을 통해 엔비디아와의 파트너십을 강화하고 매출 다변화를 꾀한다.
---
반도체 공급망 다변화는 지정학적 리스크를 관리하는 핵심 전략으로 자리 잡고 있다.

[After Rerank]
 HBM 채택 확대는 LLM 추론 효율과 메모리 대역폭 병목을 동시에 개선한다.
---
삼성전자 메모리 부문은 DDR5 전환과 HBM 수요 확대에 힘입어 수익성이 회복되고 있다.
---
SK하이닉스는 HBM3E 양산을 통해 엔비디아와의 파트너십을 강화하고 매출 다변화를 꾀한다.
---
반도체 공급망 다변화는 지정학적 리스크를 관리하는 핵심 전략으로 자리 잡고 있다.
---
원/달러 환율 상승은 반도체 수출 채산성에 단기적으로 긍정적 영향을 준다.



## 10) 생성(압축 전)

리랭크 상위 3개 문맥을 그대로 LLM에 주고 간단 답변 생성


In [38]:
# OpenAI API를 이용해 생성 파트를 수행하는 버전
# 사전 준비: pip install openai, OPENAI_API_KEY 환경변수 설정 필요

from openai import OpenAI
import os

 # Google Colab Secrets 사용 시
# from google.colab import userdata
# api_key = userdata.get('OPENAI_API_KEY')

client = OpenAI()# (api_key=api_key)

context_raw = "\n\n".join(cand_rr[:3])

prompt = (
    "다음은 사용자의 질문에 답하기 위한 문맥입니다.\n"
    "문맥을 읽고 질문에 대한 간단하고 핵심적인 답변을 작성하세요.\n\n"
    f"문맥:\n{context_raw}\n\n"
    f"질문: {demo_q['q']}\n\n"
    "정답:"
)

# OpenAI ChatCompletion으로 답변 생성
response = client.chat.completions.create(
    model="gpt-4o-mini",  # or gpt-4-turbo, gpt-4o 등 선택 가능
    messages=[
        {"role": "system", "content": "당신은 금융 분야 전문 AI 어시스턴트입니다."},
        {"role": "user", "content": prompt}
    ],
    temperature=0.3,
    max_tokens=1024
)

ans_raw = response.choices[0].message.content.strip()
print("[압축 전 답변]\n", ans_raw)


[압축 전 답변]
 HBM 채택 확대는 LLM 추론 효율을 개선하여 성능을 향상시킵니다.


In [19]:
from openai import OpenAI
import os

# 혹은 아래와 같은 방식 가능
# os.environ["OPENAI_API_KEY"] =userdata.get('OPENAI_API_KEY')

def gen(prompt, max_length=512, model="gpt-4o-mini"):
    client = OpenAI()
    response = client.responses.create(
        model=model,
        input=prompt,
        max_output_tokens=max_length,
    )

    generated = response.output_text

    return [{"generated_text": generated}]



## 11) LLMLingua 컨텍스트 압축

target_token 또는 compression_rate 로 컨텍스트 토큰 수를 줄여 비용/지연 절감
keep_keywords 로 도메인 핵심 용어 보존


In [20]:
# GPU Out of Memory 발생 시 아래 코드 활용
# Python 레퍼런스 삭제
import gc
import torch

for obj in [v for v in globals().values() if hasattr(v, 'to')]:
    del obj

gc.collect()
torch.cuda.empty_cache()


In [40]:

import torch
from llmlingua import PromptCompressor

torch.set_default_dtype(torch.float32)
compressor = PromptCompressor()
compressed = compressor.compress_prompt(
    context=[context_raw],   # 반드시 리스트로 감싸기
    instruction=f"다음 문맥은 질문 `{demo_q['q']}` 에 답하기 위한 근거다. 핵심 사실만 남기고 불필요한 서술을 제거하라.",
    question=demo_q['q'],    # 질문은 question 인자로 넣기
    target_token=10         # 토큰 목표치
)
context_cmp = compressed["compressed_prompt"]
print("압축 전 토큰 추정:", len(context_raw.split()))
print("압축 후 토큰 추정:", len(context_cmp.split()))


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

압축 전 토큰 추정: 33
압축 후 토큰 추정: 37


In [27]:

prompt_cmp = f"문맥을 바탕으로 질문에 자세하고 핵심적인 답해줘.\n문맥:\n{context_cmp}\n\n질문:{demo_q['q']}\n정답:"
ans_cmp = gen(prompt_cmp, max_length=1024)[0]['generated_text']
print("[압축 후 답변]\n", ans_cmp[:600])


[압축 후 답변]
 HBM 채택 확대는 LLM 추론 성능과 메모리 대역폭 병목을 개선한다.



## 12) RAGAS 평가(라이트)

Retrieval: context_precision, context_recall
E2E: answer_semantic_similarity (로컬 임베딩 기반)
주의: 버전에 따라 임베딩 백엔드 설정이 필요할 수 있음


In [30]:
!pip install -U ragas

#아래 코드는 ragas 최신 버전에서 동작, 다만 colab에서는 ragas==0.3.9만 지원하기에 코드는 참고

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


INFO: pip is looking at multiple versions of langchain-openai to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of langchain-openai to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
  Using cached langchain_community-0.4.1-py3-none-any.whl.metadata (3.0 kB)
INFO: pip is looking at multiple versions of langchain-community to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of langchain-community to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might ne

In [ ]:
import os
from datasets import Dataset
from ragas import evaluate
from ragas.metrics import (
    context_precision,
    context_recall,
    answer_relevance,
)
from ragas.llms import OpenAI as RagasOpenAI
from ragas.embeddings import OpenAIEmbeddings
from openai import OpenAI

client = OpenAI()

ragas_llm = RagasOpenAI(
    model="gpt-4o-mini",
    api_key=os.environ["OPENAI_API_KEY"],
)
ragas_embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small",
    api_key=os.environ["OPENAI_API_KEY"],
)

def gen(prompt):
    res = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}],
        max_tokens=1024,
    )
    return res.choices[0].message.content

samples = []
for qobj in queries[:3]:
    q_txt = qobj["q"]

    hyb = hybrid_scores(embed_model, index_manual, q_txt, lam=0.5)
    order = np.argsort(hyb)[::-1][:3]
    ctx_list = [docs_texts[i] for i in order]

    ctx_join = "\n\n".join(ctx_list)
    prompt = f"맥락:\n{ctx_join}\n\n질문:{q_txt}\n답변:"
    ans = gen(prompt)

    gt = " ".join(qobj.get("needs", {}).get("tags", []))

    samples.append(
        {
            "question": q_txt,
            "contexts": ctx_list,
            "answer": ans,
            "ground_truth": gt,
        }
    )

ds = Dataset.from_list(samples)

result = evaluate(
    ds,
    metrics=[
        context_precision,
        context_recall,
        answer_relevance,
    ],
    llm=ragas_llm,
    embeddings=ragas_embeddings,
)

result.to_pandas()



## 13) 임베딩 A/B 비교 (NMIXX vs Baseline 폴백)

동일 코퍼스/질의/지표(P@k, R@k, MRR, nDCG@k)로 두 임베딩을 비교
NMIXX가 없으면 자동으로 BGE vs MiniLM 비교로 폴백


In [33]:

from collections import OrderedDict
from sentence_transformers import SentenceTransformer

def build_index_for(model_id):
    "특정 모델 ID에 대해 임베딩/인덱스 구성"
    m = SentenceTransformer(model_id)
    idx, vec = build_faiss(m, docs_texts)
    return m, idx, vec

ab_candidates = []
try:
    _m, _idx, _vec = build_index_for(NMIXX_HUB_ID)
    ab_candidates.append(NMIXX_HUB_ID)
except Exception as e:
    print("[WARN] NMIXX 로드 실패:", e)

if "BAAI/bge-base-en-v1.5" not in ab_candidates:
    ab_candidates.append("BAAI/bge-base-en-v1.5")
if len(ab_candidates) < 2:
    ab_candidates.append("sentence-transformers/all-MiniLM-L6-v2")

print("A/B 대상:", ab_candidates[:2])

models_ab = OrderedDict()
for mid in ab_candidates[:2]:
    print("[BUILD]", mid)
    m, idx, vec = build_index_for(mid)
    models_ab[mid] = {"model": m, "index": idx, "vec": vec}


No sentence-transformers model found with name nmixx-fin/nmixx-bge-m3. Creating a new one with mean pooling.


A/B 대상: ['nmixx-fin/nmixx-bge-m3', 'BAAI/bge-base-en-v1.5']
[BUILD] nmixx-fin/nmixx-bge-m3


No sentence-transformers model found with name nmixx-fin/nmixx-bge-m3. Creating a new one with mean pooling.


[BUILD] BAAI/bge-base-en-v1.5


In [34]:

import pandas as pd
import numpy as np

def eval_ab(models_ab, lam=0.5, k=5, use_rewrite=True):
    "각 모델에 대해 vector/hybrid 성능을 측정하고 평균 비교"
    table = []
    for mid, pack in models_ab.items():
        m, idx = pack["model"], pack["index"]
        rows = []
        for qobj in queries:
            q = qobj["q"]; needs = qobj["needs"]
            subs = rule_rewrite(q) if use_rewrite else [q]
            vec = np.zeros(len(docs_texts)); bm = np.zeros(len(docs_texts))
            for sub in subs:
                vec = np.maximum(vec, cosine_scores(m, idx, sub))
                bm  = np.maximum(bm, bm25_scores(sub))
            order,_ = topk_by_scores(vec, k=k); rows.append(metrics_at_k(order, needs, k=k) | {"mode":"vector"})
            hyb = lam*((bm-bm.min())/(bm.ptp()+1e-9)) + (1-lam)*((vec-vec.min())/(vec.ptp()+1e-9))
            order,_ = topk_by_scores(hyb, k=k); rows.append(metrics_at_k(order, needs, k=k) | {"mode":f"hybrid_{lam}"})
        df = pd.DataFrame(rows).assign(model=mid)
        agg = df.groupby(["model","mode"]).mean(numeric_only=True).reset_index()
        table.append(agg)
    return pd.concat(table, ignore_index=True)

df_ab = eval_ab(models_ab, lam=0.5, k=5, use_rewrite=True)
df_ab.sort_values(["mode","model"])


,model,mode,P@5,R@5,MRR,nDCG@5
2,BAAI/bge-base-en-v1.5,hybrid_0.5,0.32,1.0,1.0,0.977438
0,nmixx-fin/nmixx-bge-m3,hybrid_0.5,0.32,1.0,1.0,0.973325
3,BAAI/bge-base-en-v1.5,vector,0.32,1.0,0.8,0.858365
1,nmixx-fin/nmixx-bge-m3,vector,0.32,1.0,1.0,1.000000
